This program produces a Shiny app that simulates the Reed-Frost model of a disease spreading through a population. 

In [ ]:
library(shiny)
library(igraph)

ui <- fluidPage(
  titlePanel("Reed-Frost Simulation"),
  h4("The Reed-Frost model is a simple epidemiologic model in which each individual in a population 
     takes one of three possible states: Susceptible (S), Infected (I), or Recovered (R)."),
  br(),
  h4("At each time step, every infected (I) individual has some probability p of coming into contact with each 
     susceptible (S) person, independently of the others. If a susceptible individual comes into 
     contact with at least one infected individual, they will become infected for the next time step 
     and recovered (R) the time step after that. As such, each infected individual is only infected for 
     one time step and cannot be re-infected after they reach 'recovered' status."),
  br(),
  h4("Specify the number of people in the population, the number of people initially infected, and the
     probability of contact between each infected and susceptible person. Press 'Begin Simulation' to 
     set the simulation to your starting parameters and press 'Next Step' to simulate one step."),
  br(),
  sidebarLayout(
    sidebarPanel(
      numericInput("popsize", "Size of Population:",
                   value=150),
      numericInput("infections", "Initial infections:",
                   value = 1),
      numericInput("p", "Probability of contact:",
                   value = 0.03),
      actionButton("start", "Begin Simulation"),
      actionButton("step", "Next Step")
    ),
    mainPanel(
      plotOutput("graph")
    )
  )
)

server <- function(input, output) {
    
  # x$g will be the graph we're modifying. It's initialized to have 150 vertices because
  # that's the arbitrary starting value set in the parameters.
  # Once the user clicks the start button, the graph will be set to whatever parameters they inputted.
  x <- reactiveValues(g=graph(edges=c(), n=150, directed=F))
  
  ## Set initial graph w/ the specified parameters when "Begin Simulation" button is pressed
  setGraph <- observeEvent(input$start, {
    x$g <- graph(edges=c(), n=input$popsize, directed=F)
    Status <- c() #vector representing the status of each person in the population
    for (i in 1:(input$infections)) { #set the first I0 people to "infected"
      Status <- c(Status, "I")
    }
    for (i in 1:(input$popsize - input$infections)) { #set the rest of the population to "susceptible"
      Status <- c(Status, "S")
    }
    V(x$g)$status <- Status #assign the elements of Status to the nodes in the graph x$g
  })
  
  ## update the graph when the "Next Step" button is clicked
  takeStep <- observeEvent(input$step, {
    if (input$start) { #if the "Begin Simulation" button has already been clicked
      Status <- V(x$g)$status
      newStatus <- Status
      for (i in 1:length(Status)) { #for each node in the population
        if (Status[i] == "I") { #if they are infected
          newStatus[i] = "R" #set their new status to recovered
          for (j in 1:length(Status)) { #traverse the population again
            if (Status[j] == "S") { #if they are susceptible
              u <- runif(1) #create a u ~ Unif(0, 1) random variable
              if (u < input$p) { #if u is less than the p that the user specified
                x$g <- x$g + edge(i,j) #draw an edge between the infected node and the susceptible node
                if (newStatus[j] != "I") { #if node j hasn't already been infected this time step
                  newStatus[j] = "I" #set its new status to infected
                }
              }
            }
          }
        }
      }
      Status <- newStatus
      V(x$g)$status = Status
    }
  })
  
  ## plot the graph
  output$graph <- renderPlot({
      plot(x$g, vertex.size=7, vertex.label=NA,
           vertex.color=c("green", "red", "gray")[1+(V(x$g)$status=="I")+(2*(V(x$g)$status=="R"))])
      legend(1, 1, legend=c("Susceptible", "Infected", "Recovered"), pch=19, col=c("green", "red", "gray"))
  })
}


shinyApp(ui=ui, server=server) #run the app